# 🔀 Week 16 - Day 2: LangGraph Workflows

## Today's Goals:
✅ Master conditional edges for dynamic routing

✅ Build parallel execution workflows

✅ Create multi-agent systems

✅ Implement inter-agent communication

✅ Orchestrate complex tool workflows

---

## 🔧 Part 1: Setup - Install & Import All Libraries

**IMPORTANT:** Run ALL cells in this part sequentially!

In [3]:
# STEP 1: Install required packages
print("📦 Installing packages... (this may take 1-2 minutes)\n")

# IMPORTANT: Install with compatible versions to avoid conflicts
!pip install -q langchain-core==0.3.29 langchain==0.3.14 langchain-community==0.3.14
!pip install -q langgraph==0.2.61 langchain-groq==0.2.3
!pip install -q python-dotenv

print("\n✅ All packages installed successfully!")
print("\n⚠️ NOTE: If you see version warnings above, they can be ignored.")
print("The packages will still work correctly for this notebook.")

📦 Installing packages... (this may take 1-2 minutes)



ERROR: Could not install packages due to an OSError: [('c:\\users\\zigron\\anaconda3\\envs\\ai_bootcamp\\lib\\site-packages\\langchain_core\\tools\\__pycache__\\base.cpython-311.pyc.1991909913712', 'C:\\Users\\Zigron\\anaconda3\\envs\\ai_bootcamp\\Lib\\site-packages\\langchain_core\\~ools\\__pycache__\\base.cpython-311.pyc.1991909913712', "[Errno 2] No such file or directory: 'c:\\\\users\\\\zigron\\\\anaconda3\\\\envs\\\\ai_bootcamp\\\\lib\\\\site-packages\\\\langchain_core\\\\tools\\\\__pycache__\\\\base.cpython-311.pyc.1991909913712'")]

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.3.80 which is incompatible.
langchain 0.1.20 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.4.49 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>


✅ All packages installed successfully!

⚠️ NOTE: If you see version warnings above, they can be ignored.
The packages will still work correctly for this notebook.


In [1]:
# STEP 2: Import ALL libraries
import os
import json
import warnings
warnings.filterwarnings('ignore')

# LangGraph Core
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent, ToolNode
from langgraph.graph.message import add_messages

# LangChain Core
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# LLM
from langchain_groq import ChatGroq

# Type hints
from typing import Annotated, TypedDict, Literal, List, Optional
import operator

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [2]:
# STEP 3: Set up API Key for Groq
# Get your free API key from: https://console.groq.com/

GROQ_API_KEY = "your-groq-api-key-here"  # Replace with your actual key

os.environ["GROQ_API_KEY"] = GROQ_API_KEY

# Initialize our LLM
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=1000
)

print("✅ LLM initialized!")
print("🚀 Ready to build advanced workflows!")

✅ LLM initialized!
🚀 Ready to build advanced workflows!


---

## 🔀 Part 2: Conditional Edges - Dynamic Routing

Yesterday we learned about basic edges. Today, we'll master **conditional edges** - the key to making agents that can make decisions!

### What are Conditional Edges?

```
┌─────────────────────────────────────────────────────┐
│                                                     │
│   Regular Edge:     A ──────────────► B             │
│                     (always goes to B)              │
│                                                     │
│   Conditional Edge: A ──?──┬──► B                   │
│                            │                        │
│                            └──► C                   │
│                     (decides based on condition)    │
│                                                     │
└─────────────────────────────────────────────────────┘
```

In [3]:
# Let's build a simple router that directs queries to different handlers

# Define the state
class RouterState(TypedDict):
    """State for our routing example"""
    messages: Annotated[list, add_messages]
    query_type: str  # Will be: "math", "text", or "general"

print("✅ RouterState defined!")
print("\n💡 Notice we added 'query_type' to track what kind of query we have.")

✅ RouterState defined!

💡 Notice we added 'query_type' to track what kind of query we have.


In [4]:
# Create a classifier node that determines the query type

def classifier_node(state: RouterState):
    """Classifies the query into: math, text, or general"""
    last_message = state["messages"][-1].content.lower()
    
    # Simple keyword-based classification
    math_keywords = ["add", "subtract", "multiply", "divide", "calculate", "sum", "plus", "minus"]
    text_keywords = ["reverse", "length", "count", "vowels", "uppercase", "lowercase", "spell"]
    
    if any(keyword in last_message for keyword in math_keywords):
        query_type = "math"
    elif any(keyword in last_message for keyword in text_keywords):
        query_type = "text"
    else:
        query_type = "general"
    
    print(f"🔍 Classifier: Detected query type = '{query_type}'")
    return {"query_type": query_type}

print("✅ Classifier node defined!")

✅ Classifier node defined!


In [5]:
# Create specialized handler nodes

def math_handler(state: RouterState):
    """Handles math-related queries"""
    print("🔢 Math Handler: Processing math query...")
    response = llm.invoke([
        SystemMessage(content="You are a math expert. Solve the math problem step by step."),
        state["messages"][-1]
    ])
    return {"messages": [response]}

def text_handler(state: RouterState):
    """Handles text-related queries"""
    print("📝 Text Handler: Processing text query...")
    response = llm.invoke([
        SystemMessage(content="You are a text processing expert. Help with the text operation."),
        state["messages"][-1]
    ])
    return {"messages": [response]}

def general_handler(state: RouterState):
    """Handles general queries"""
    print("💬 General Handler: Processing general query...")
    response = llm.invoke([
        SystemMessage(content="You are a helpful assistant. Answer the question concisely."),
        state["messages"][-1]
    ])
    return {"messages": [response]}

print("✅ Handler nodes defined!")

✅ Handler nodes defined!


In [6]:
# The routing function - decides where to go based on query_type

def route_query(state: RouterState) -> Literal["math_handler", "text_handler", "general_handler"]:
    """Routes to the appropriate handler based on query type"""
    query_type = state["query_type"]
    
    if query_type == "math":
        return "math_handler"
    elif query_type == "text":
        return "text_handler"
    else:
        return "general_handler"

print("✅ Routing function defined!")
print("\n💡 This function returns the NAME of the node to go to next.")

✅ Routing function defined!

💡 This function returns the NAME of the node to go to next.


In [7]:
# Build the router graph

router_graph = StateGraph(RouterState)

# Add all nodes
router_graph.add_node("classifier", classifier_node)
router_graph.add_node("math_handler", math_handler)
router_graph.add_node("text_handler", text_handler)
router_graph.add_node("general_handler", general_handler)

# Add edges
router_graph.add_edge(START, "classifier")  # Start -> Classifier

# Add CONDITIONAL edge from classifier
router_graph.add_conditional_edges(
    "classifier",      # From this node
    route_query,       # Use this function to decide
    {                  # Map return values to node names
        "math_handler": "math_handler",
        "text_handler": "text_handler",
        "general_handler": "general_handler"
    }
)

# All handlers go to END
router_graph.add_edge("math_handler", END)
router_graph.add_edge("text_handler", END)
router_graph.add_edge("general_handler", END)

# Compile
router_app = router_graph.compile()

print("✅ Router graph compiled!")
print("\n📊 Graph Structure:")
print("   START → classifier → [math|text|general]_handler → END")

✅ Router graph compiled!

📊 Graph Structure:
   START → classifier → [math|text|general]_handler → END


In [8]:
# Test the router with different queries!

def test_router(query):
    print(f"\n{'='*60}")
    print(f"❓ Query: {query}")
    print("="*60)
    
    result = router_app.invoke({
        "messages": [HumanMessage(content=query)],
        "query_type": ""  # Will be set by classifier
    })
    
    print(f"\n🤖 Response: {result['messages'][-1].content}")
    return result

In [9]:
# Test 1: Math query
test_router("Can you add 25 and 37?")


❓ Query: Can you add 25 and 37?
🔍 Classifier: Detected query type = 'math'
🔢 Math Handler: Processing math query...


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [ ]:
# Test 2: Text query
test_router("How many vowels are in 'artificial intelligence'?")

In [ ]:
# Test 3: General query
test_router("What is the capital of France?")

### 💡 Key Insights - Conditional Edges:

✅ **add_conditional_edges()** takes: source node, routing function, and mapping

✅ **Routing function** returns the NAME of the next node

✅ **Mapping** connects return values to actual node names

✅ This enables **dynamic decision-making** in your workflows!

---

## 🔄 Part 3: Building an Agent with Tool Routing

Now let's build a more sophisticated agent that uses conditional edges to decide whether to use tools or respond directly.

In [ ]:
# First, let's create some tools

@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression. Use this for any math calculations.
    
    Args:
        expression: A mathematical expression like '2 + 2' or '10 * 5'
    """
    try:
        # Safe evaluation of math expressions
        allowed_chars = set('0123456789+-*/(). ')
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return f"Result: {result}"
        else:
            return "Error: Invalid characters in expression"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def get_weather(city: str) -> str:
    """Get the current weather for a city. Use this when asked about weather.
    
    Args:
        city: The name of the city
    """
    # Simulated weather data
    weather_data = {
        "new york": "Sunny, 72°F",
        "london": "Cloudy, 58°F",
        "tokyo": "Rainy, 65°F",
        "paris": "Partly Cloudy, 68°F",
        "sydney": "Sunny, 78°F"
    }
    city_lower = city.lower()
    return weather_data.get(city_lower, f"Weather data not available for {city}")

@tool
def search_info(query: str) -> str:
    """Search for information about a topic. Use for factual questions.
    
    Args:
        query: The search query
    """
    # Simulated search results
    info_db = {
        "python": "Python is a high-level programming language known for its simplicity.",
        "ai": "AI (Artificial Intelligence) is the simulation of human intelligence by machines.",
        "langgraph": "LangGraph is a framework for building stateful AI agents with graph-based workflows."
    }
    query_lower = query.lower()
    for key, value in info_db.items():
        if key in query_lower:
            return value
    return f"No specific information found for: {query}"

tools = [calculate, get_weather, search_info]
print("✅ Tools created: calculate, get_weather, search_info")

In [ ]:
# Define the agent state

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

print("✅ Agent state and LLM with tools ready!")

In [ ]:
# Define the agent node

def agent_node(state: AgentState):
    """The agent decides what to do"""
    print("🤖 Agent: Thinking...")
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# Create tool node
tool_node = ToolNode(tools)

print("✅ Agent and Tool nodes defined!")

In [ ]:
# The key routing function - should we use tools or finish?

def should_use_tools(state: AgentState) -> Literal["tools", "end"]:
    """Decides whether to call tools or end the conversation"""
    last_message = state["messages"][-1]
    
    # If the LLM made tool calls, go to tools
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        print(f"🔧 Router: Tool calls detected -> going to tools")
        return "tools"
    
    # Otherwise, we're done
    print("✅ Router: No tool calls -> ending")
    return "end"

print("✅ Routing function defined!")

In [ ]:
# Build the agent graph

agent_graph = StateGraph(AgentState)

# Add nodes
agent_graph.add_node("agent", agent_node)
agent_graph.add_node("tools", tool_node)

# Add edges
agent_graph.add_edge(START, "agent")

# Conditional edge from agent
agent_graph.add_conditional_edges(
    "agent",
    should_use_tools,
    {
        "tools": "tools",
        "end": END
    }
)

# Tools always go back to agent (the loop!)
agent_graph.add_edge("tools", "agent")

# Compile
smart_agent = agent_graph.compile()

print("✅ Smart Agent compiled!")
print("\n📊 Graph Structure:")
print("   START → agent ⟷ tools")
print("              ↓")
print("             END")

In [ ]:
# Helper function to test the agent

def ask_smart_agent(question):
    print(f"\n{'='*60}")
    print(f"❓ Question: {question}")
    print("="*60)
    
    result = smart_agent.invoke({
        "messages": [HumanMessage(content=question)]
    })
    
    # Get final response
    final_response = result["messages"][-1].content
    print(f"\n🤖 Final Answer: {final_response}")
    return result

In [ ]:
# Test with a calculation
ask_smart_agent("What is 125 * 8?")

In [ ]:
# Test with weather
ask_smart_agent("What's the weather like in Tokyo?")

In [ ]:
# Test with general question (no tool needed)
ask_smart_agent("Say hello in French")

---

## 👥 Part 4: Multi-Agent Systems

Now for the exciting part - building systems with **multiple specialized agents** that work together!

### Architecture: Supervisor Pattern

```
┌─────────────────────────────────────────────────────┐
│                                                     │
│                   👨‍💼 SUPERVISOR                     │
│                   (Decides who works)               │
│                         │                           │
│           ┌─────────────┼─────────────┐             │
│           ▼             ▼             ▼             │
│      🔬 Research   💻 Coder    📝 Writer            │
│        Agent        Agent       Agent               │
│                                                     │
└─────────────────────────────────────────────────────┘
```

In [10]:
# Define state for multi-agent system

class MultiAgentState(TypedDict):
    messages: Annotated[list, add_messages]
    next_agent: str  # Who should work next
    task_complete: bool  # Is the task done?

print("✅ Multi-Agent State defined!")

✅ Multi-Agent State defined!


In [11]:
# Create specialized agent prompts

researcher_prompt = """You are a Research Agent. Your job is to:
- Gather and analyze information
- Provide facts and data
- Be thorough but concise

Respond with your research findings."""

coder_prompt = """You are a Coding Agent. Your job is to:
- Write clean, working code
- Explain code when needed
- Fix bugs and optimize

Respond with code solutions."""

writer_prompt = """You are a Writing Agent. Your job is to:
- Create clear, engaging content
- Summarize information
- Format text professionally

Respond with polished writing."""

print("✅ Agent prompts defined!")

✅ Agent prompts defined!


In [12]:
# Create the specialized agent nodes

def researcher_node(state: MultiAgentState):
    """Research agent processes the request"""
    print("\n🔬 Research Agent: Working...")
    
    messages = [
        SystemMessage(content=researcher_prompt),
        HumanMessage(content=state["messages"][-1].content if isinstance(state["messages"][-1], HumanMessage) 
                     else f"Based on previous work: {state['messages'][-1].content}")
    ]
    
    response = llm.invoke(messages)
    print(f"   📊 Research complete!")
    
    return {
        "messages": [AIMessage(content=f"[Research Agent]: {response.content}")]
    }

def coder_node(state: MultiAgentState):
    """Coder agent processes the request"""
    print("\n💻 Coder Agent: Working...")
    
    messages = [
        SystemMessage(content=coder_prompt),
        HumanMessage(content=state["messages"][-1].content if isinstance(state["messages"][-1], HumanMessage)
                     else f"Based on previous work: {state['messages'][-1].content}")
    ]
    
    response = llm.invoke(messages)
    print(f"   💾 Code complete!")
    
    return {
        "messages": [AIMessage(content=f"[Coder Agent]: {response.content}")]
    }

def writer_node(state: MultiAgentState):
    """Writer agent processes the request"""
    print("\n📝 Writer Agent: Working...")
    
    messages = [
        SystemMessage(content=writer_prompt),
        HumanMessage(content=state["messages"][-1].content if isinstance(state["messages"][-1], HumanMessage)
                     else f"Based on previous work: {state['messages'][-1].content}")
    ]
    
    response = llm.invoke(messages)
    print(f"   ✍️ Writing complete!")
    
    return {
        "messages": [AIMessage(content=f"[Writer Agent]: {response.content}")]
    }

print("✅ Specialized agent nodes defined!")

✅ Specialized agent nodes defined!


In [13]:
# Create the Supervisor node

supervisor_prompt = """You are a Supervisor managing a team of agents:
- researcher: For gathering information, facts, data analysis
- coder: For writing code, debugging, technical solutions
- writer: For creating content, summarizing, formatting text
- FINISH: When the task is complete

Based on the user's request and any previous work, decide who should work next.
Respond with ONLY one word: 'researcher', 'coder', 'writer', or 'FINISH'.
"""

def supervisor_node(state: MultiAgentState):
    """Supervisor decides which agent should work next"""
    print("\n👨‍💼 Supervisor: Analyzing task...")
    
    # Build context from all messages
    context = "\n".join([f"{type(m).__name__}: {m.content}" for m in state["messages"]])
    
    messages = [
        SystemMessage(content=supervisor_prompt),
        HumanMessage(content=f"Current state:\n{context}\n\nWho should work next?")
    ]
    
    response = llm.invoke(messages)
    decision = response.content.strip().lower()
    
    # Clean up the decision
    if "finish" in decision:
        decision = "FINISH"
    elif "research" in decision:
        decision = "researcher"
    elif "code" in decision:
        decision = "coder"
    elif "writ" in decision:
        decision = "writer"
    else:
        decision = "FINISH"  # Default to finish if unclear
    
    print(f"   📋 Decision: {decision}")
    
    return {
        "next_agent": decision,
        "task_complete": decision == "FINISH"
    }

print("✅ Supervisor node defined!")

✅ Supervisor node defined!


In [14]:
# Routing function for supervisor

def route_supervisor(state: MultiAgentState) -> Literal["researcher", "coder", "writer", "end"]:
    """Route based on supervisor's decision"""
    next_agent = state.get("next_agent", "FINISH")
    
    if next_agent == "FINISH" or state.get("task_complete", False):
        return "end"
    elif next_agent == "researcher":
        return "researcher"
    elif next_agent == "coder":
        return "coder"
    elif next_agent == "writer":
        return "writer"
    else:
        return "end"

print("✅ Supervisor routing function defined!")

✅ Supervisor routing function defined!


In [15]:
# Build the multi-agent graph

multi_agent_graph = StateGraph(MultiAgentState)

# Add all nodes
multi_agent_graph.add_node("supervisor", supervisor_node)
multi_agent_graph.add_node("researcher", researcher_node)
multi_agent_graph.add_node("coder", coder_node)
multi_agent_graph.add_node("writer", writer_node)

# Start goes to supervisor
multi_agent_graph.add_edge(START, "supervisor")

# Supervisor routes conditionally
multi_agent_graph.add_conditional_edges(
    "supervisor",
    route_supervisor,
    {
        "researcher": "researcher",
        "coder": "coder",
        "writer": "writer",
        "end": END
    }
)

# All agents report back to supervisor
multi_agent_graph.add_edge("researcher", "supervisor")
multi_agent_graph.add_edge("coder", "supervisor")
multi_agent_graph.add_edge("writer", "supervisor")

# Compile
multi_agent_app = multi_agent_graph.compile()

print("✅ Multi-Agent System compiled!")
print("\n📊 Graph Structure:")
print("   START → supervisor ⟷ [researcher|coder|writer]")
print("                ↓")
print("               END")

✅ Multi-Agent System compiled!

📊 Graph Structure:
   START → supervisor ⟷ [researcher|coder|writer]
                ↓
               END


In [16]:
# Test the multi-agent system

def run_multi_agent(task):
    print(f"\n{'='*70}")
    print(f"📋 Task: {task}")
    print("="*70)
    
    result = multi_agent_app.invoke({
        "messages": [HumanMessage(content=task)],
        "next_agent": "",
        "task_complete": False
    })
    
    print(f"\n{'='*70}")
    print("📦 Final Results:")
    print("="*70)
    for msg in result["messages"]:
        if isinstance(msg, AIMessage):
            print(f"\n{msg.content[:500]}...") if len(msg.content) > 500 else print(f"\n{msg.content}")
    
    return result

In [17]:
# Test 1: Research task
run_multi_agent("Research the benefits of Python for data science")


📋 Task: Research the benefits of Python for data science

👨‍💼 Supervisor: Analyzing task...


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [18]:
# Test 2: Coding task
run_multi_agent("Write a Python function to calculate factorial")


📋 Task: Write a Python function to calculate factorial

👨‍💼 Supervisor: Analyzing task...


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [ ]:
# Test 3: Writing task
run_multi_agent("Write a short introduction about AI agents")

---

## 🔧 Part 5: Tool Orchestration

Let's create a more advanced agent that can orchestrate multiple tools in sequence to solve complex problems.

In [ ]:
# Create tools for a task management system

# Simulated task database
task_db = {}
task_counter = 0

@tool
def create_task(title: str, description: str) -> str:
    """Create a new task with a title and description.
    
    Args:
        title: The title of the task
        description: A description of what needs to be done
    """
    global task_counter
    task_counter += 1
    task_id = f"TASK-{task_counter}"
    task_db[task_id] = {
        "title": title,
        "description": description,
        "status": "pending",
        "priority": "medium"
    }
    return f"Created task {task_id}: {title}"

@tool
def list_tasks() -> str:
    """List all tasks in the system."""
    if not task_db:
        return "No tasks found."
    
    result = "Current Tasks:\n"
    for task_id, task in task_db.items():
        result += f"- {task_id}: {task['title']} [{task['status']}]\n"
    return result

@tool
def update_task_status(task_id: str, status: str) -> str:
    """Update the status of a task.
    
    Args:
        task_id: The ID of the task (e.g., TASK-1)
        status: New status (pending, in_progress, completed)
    """
    if task_id not in task_db:
        return f"Task {task_id} not found."
    
    task_db[task_id]["status"] = status
    return f"Updated {task_id} status to: {status}"

@tool
def set_priority(task_id: str, priority: str) -> str:
    """Set the priority of a task.
    
    Args:
        task_id: The ID of the task (e.g., TASK-1)
        priority: Priority level (low, medium, high, urgent)
    """
    if task_id not in task_db:
        return f"Task {task_id} not found."
    
    task_db[task_id]["priority"] = priority
    return f"Set {task_id} priority to: {priority}"

task_tools = [create_task, list_tasks, update_task_status, set_priority]

print("✅ Task management tools created!")
print("   📝 create_task, list_tasks, update_task_status, set_priority")

In [ ]:
# Create a task management agent

task_agent = create_react_agent(
    model=llm,
    tools=task_tools
)

print("✅ Task Management Agent created!")

In [ ]:
# Helper function

def manage_tasks(request):
    print(f"\n{'='*60}")
    print(f"📋 Request: {request}")
    print("="*60)
    
    result = task_agent.invoke({
        "messages": [HumanMessage(content=request)]
    })
    
    # Show the conversation
    for msg in result["messages"]:
        if isinstance(msg, AIMessage):
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                for tc in msg.tool_calls:
                    print(f"\n🔧 Tool: {tc['name']}({tc['args']})")
            elif msg.content:
                print(f"\n🤖 Agent: {msg.content}")
        elif hasattr(msg, 'content') and 'ToolMessage' in str(type(msg)):
            print(f"   📤 Result: {msg.content}")
    
    return result

In [ ]:
# Test: Create multiple tasks
manage_tasks("Create a task called 'Learn LangGraph' with description 'Complete the tutorial'")

In [ ]:
# Create another task
manage_tasks("Create a task 'Build AI Agent' with description 'Implement multi-agent system'")

In [ ]:
# List all tasks
manage_tasks("Show me all tasks")

In [ ]:
# Complex request: Multiple operations
manage_tasks("Set TASK-1 to high priority and mark it as in_progress")

---

## 🎯 Part 6: Mini Challenge

### 🏆 Challenge: Build a Customer Support Multi-Agent System!

Create a multi-agent system with:
1. **Triage Agent** - Classifies customer issues (billing, technical, general)
2. **Billing Agent** - Handles payment and subscription questions
3. **Technical Agent** - Handles product/technical issues
4. **Supervisor** - Routes between agents

**Bonus:** Add tools for looking up customer info!

In [ ]:
# Your code here!
# Hint: Follow the multi-agent pattern from Part 4

# Step 1: Define the state
# class CustomerSupportState(TypedDict):
#     messages: Annotated[list, add_messages]
#     issue_type: str
#     ...

# Step 2: Create specialized agents
# def billing_agent(state): ...
# def technical_agent(state): ...

# Step 3: Create supervisor
# def supervisor(state): ...

# Step 4: Build the graph
# ...

pass

In [ ]:
# Solution Skeleton (Expand this!)

class CustomerSupportState(TypedDict):
    messages: Annotated[list, add_messages]
    issue_type: str
    resolved: bool

def triage_agent(state: CustomerSupportState):
    """Classifies the customer issue"""
    print("🏥 Triage: Analyzing issue...")
    message = state["messages"][-1].content.lower()
    
    if any(word in message for word in ["bill", "payment", "charge", "subscription", "refund"]):
        issue_type = "billing"
    elif any(word in message for word in ["error", "bug", "crash", "not working", "broken"]):
        issue_type = "technical"
    else:
        issue_type = "general"
    
    print(f"   Issue type: {issue_type}")
    return {"issue_type": issue_type}

def billing_agent(state: CustomerSupportState):
    """Handles billing issues"""
    print("💳 Billing Agent: Handling billing issue...")
    response = llm.invoke([
        SystemMessage(content="You are a billing support agent. Help with payment and subscription issues."),
        state["messages"][-1]
    ])
    return {"messages": [response], "resolved": True}

def technical_agent(state: CustomerSupportState):
    """Handles technical issues"""
    print("🔧 Technical Agent: Handling technical issue...")
    response = llm.invoke([
        SystemMessage(content="You are a technical support agent. Help troubleshoot product issues."),
        state["messages"][-1]
    ])
    return {"messages": [response], "resolved": True}

def general_agent(state: CustomerSupportState):
    """Handles general inquiries"""
    print("💬 General Agent: Handling inquiry...")
    response = llm.invoke([
        SystemMessage(content="You are a friendly customer support agent. Help with general questions."),
        state["messages"][-1]
    ])
    return {"messages": [response], "resolved": True}

def route_issue(state: CustomerSupportState):
    issue_type = state.get("issue_type", "general")
    if issue_type == "billing":
        return "billing"
    elif issue_type == "technical":
        return "technical"
    else:
        return "general"

# Build the graph
support_graph = StateGraph(CustomerSupportState)
support_graph.add_node("triage", triage_agent)
support_graph.add_node("billing", billing_agent)
support_graph.add_node("technical", technical_agent)
support_graph.add_node("general", general_agent)

support_graph.add_edge(START, "triage")
support_graph.add_conditional_edges("triage", route_issue, {
    "billing": "billing",
    "technical": "technical",
    "general": "general"
})
support_graph.add_edge("billing", END)
support_graph.add_edge("technical", END)
support_graph.add_edge("general", END)

support_app = support_graph.compile()

print("✅ Customer Support System ready!")

In [ ]:
# Test the customer support system

def get_support(issue):
    print(f"\n{'='*60}")
    print(f"🎫 Customer Issue: {issue}")
    print("="*60)
    
    result = support_app.invoke({
        "messages": [HumanMessage(content=issue)],
        "issue_type": "",
        "resolved": False
    })
    
    print(f"\n🤖 Response: {result['messages'][-1].content}")
    return result

# Test billing issue
get_support("I was charged twice for my subscription last month")

In [ ]:
# Test technical issue
get_support("The app keeps crashing when I try to upload files")

---

## 📚 Summary - What We Learned Today

### 1. Conditional Edges 🔀
- Use `add_conditional_edges()` for dynamic routing
- Routing functions return the **name** of the next node
- Enable if/else logic in your workflows

### 2. Agent Tool Routing 🔧
- Agents decide whether to use tools or respond
- The classic pattern: agent → tools → agent (loop)
- Check for `tool_calls` to determine next step

### 3. Multi-Agent Systems 👥
- Supervisor pattern: one agent coordinates others
- Specialized agents handle specific domains
- Agents report back to supervisor after working

### 4. Tool Orchestration 🎭
- Agents can call multiple tools in sequence
- Tools can interact with databases/state
- Complex workflows from simple components

---

## 🎯 Key Takeaways

✅ **Conditional edges** enable dynamic, intelligent routing

✅ **Multi-agent systems** divide complex tasks among specialists

✅ **Supervisor pattern** coordinates agent collaboration

✅ **Tool orchestration** enables agents to perform real actions

✅ **LangGraph** makes building these systems straightforward!

---

## 💡 Pro Tips

1. **Keep agents focused** - One agent, one specialty
2. **Clear routing logic** - Make decisions explicit
3. **Limit iterations** - Add max loops to prevent infinite cycles
4. **Log everything** - Print statements help debug
5. **Test components** - Test each node before connecting

---

## 🚀 Next Steps - Tomorrow!

**Day 3: Communication Skills**
- Explaining agentic systems to non-technical audiences
- Managing expectations in AI projects
- Ethical communication around AI limitations

**Then: Build Your Personalized AI Learning Tutor! 🎓**

---

## 🎉 Congratulations!

You've mastered advanced LangGraph workflows!

You now know how to:
- ✅ Use conditional edges for smart routing
- ✅ Build multi-agent systems
- ✅ Implement the supervisor pattern
- ✅ Orchestrate complex tool workflows
- ✅ Create collaborative AI systems

**You're ready to build production-grade AI agents! 🚀**